# Aula 9 - Qualidade versus eficiência

Leandro Carísio Fernandes

<br>

Projeto:

O objetivo do exercício desta semana é construir alguns pipelines de busca e analisá-los em termos das seguintes métricas:

- Qualidade dos resultados: nDCG@10
- Latência (seg/query)
- USD por query assumindo utilização "perfeita": assim que terminou de processar uma query, já tem outra para ser processada
- USD/mês para deixar o sistema rodando para poucos usuários (ex: 100 queries/dia)
- Custo de indexação em USD

Iremos avaliar os pipelines no TREC-COVID.

A latência precisa ser menor que 2 segundos por query.

Considerar:

- 1,50 USD/hora por A100 ou 0,21 USD/hora por T4 ou 0,50 USD/hora por V100
- 0,03 USD/hora por CPU core
- 0,005 USD/hora por GB de CPU RAM


Dicas:
- Utilizar modelos de busca "SOTA" já treinados no MS MARCO como parte do pipeline, como o [SPLADE distil](https://huggingface.co/naver/splade-cocondenser-selfdistil) (esparso), [contriever](https://huggingface.co/facebook/contriever-msmarco) (denso), [Colbert-v2](https://github.com/stanford-futuredata/ColBERT) (denso), [miniLM](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2) (reranker), [monoT5-3B](https://huggingface.co/castorini/monot5-3b-msmarco) (reranker), [doc2query minus-minus](https://github.com/terrierteam/pyterrier_doc2query) (expansão de documentos + filtragem com reranqueador na etapa de indexação)
- Variar parametros como número de documentos retornados em cada estagio. Por exemplo, BM25 retorna 1000 documentos, um modelo denso ou esparso pode reranquea-los, e passar os top 50 para o miniLM/monoT5 fazer um ranqueamento final.

In [ ]:
# Diretório para a aula 9
aula9_dir = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula9-qualidade-vs-eficiencia/'

# Arquivo com as queries do TREC_COVID já expandidas, recuperado da aula 5
arquivo_docs_queries_expandidas = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/doc_com_queries_expandidas.pickle'

# Parâmetros usados na Aula7 para gerar o índice usando SPLADE.

# Esse conjunto de parâmetros é pro índice SPLADE com os textos originais
# nDCG de 0.7354 só splade (batch: 32 e max_seq_length: 256)
# param_splade = {
#    'agg': 'max',
#    'nome_modelo': 'naver/splade-cocondenser-selfdistil',
#    'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
#    'nome_arquivo_indice_invertido_docs': 'idx_splade_sem_doc2query_batch32_seq_256.pickle',
#    'indexar_com_doc2query': False,
#    'indexar_so_expansao': False,
#    'batch_size': 32,
#    'max_seq_length': 256,
#    'gerar_arquivo_indice': False # Se for True, gera e salva. Se for False reutiliza o arquivo já gerado
# }

# Esse conjunto de parâmetros é pro índice SPLADE considerando expansão dos docs via doc2query.
# nDCG de 0.7298 com doc2query + splade (batch: 32 e max_seq_length: 256)
param_splade = {
   'agg': 'max',
   'nome_modelo': 'naver/splade-cocondenser-selfdistil',
   'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
   'nome_arquivo_indice_invertido_docs': 'idx_splade_com_doc2query_batch32_seq_256.pickle',
   'indexar_com_doc2query': True,
   'indexar_so_expansao': False,
   'batch_size': 32,
   'max_seq_length': 256,
   'gerar_arquivo_indice': False # Se for True, gera e salva. Se for False reutiliza o arquivo já gerado
}

# Esse conjunto de parâmetros é pro índice SPLADE considerando apenas a expansão dos documentos via doc2query (ou seja, descartando o documento original)
# nDCG de 0.6193 com doc2query (desconsiderando o doc. original) + splade (batch: 32 e max_seq_length: 256)
#     -> Obs.: o BM25 só com a expansão dava 0.5225
# param_splade = {
#    'agg': 'max',
#    'nome_modelo': 'naver/splade-cocondenser-selfdistil',
#    'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
#    'nome_arquivo_indice_invertido_docs': 'idx_splade_com_doc2query_so_expansao_batch32_seq_256.pickle',
#    'indexar_com_doc2query': True,
#    'indexar_so_expansao': True,
#    'batch_size': 32,
#    'max_seq_length': 256,
#    'gerar_arquivo_indice': False
# }

# Parâmetros para reranking com InPars. O modelo usado nesse notebook é o fine-tuning do cross-encoder/ms-marco-MiniLM-L-6-v2
# com o fine-tuning feito na Aula anterior (8).
param_inpars = {
    'nome_modelo': '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula8-inpars/modelos/1neg_1pos_teste_apaga_token_type_id_no_treino/validacao',
    'nome_tokenizer': '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula8-inpars/modelos/1neg_1pos_teste_apaga_token_type_id_no_treino/validacao', #'cross-encoder/ms-marco-MiniLM-L-6-v2',
    'max_seq_length': 512,
    'batch_size': 64,
    'considerar_expansao_no_reranking': True
}

# Dados do TREC-COVID
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Wed May 10 15:30:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    42W / 300W |   3442MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers datasets -q
!pip install transformers sentence-transformers -q
!pip install pyserini -q
!pip install faiss-gpu -q

## Carrega queries e documentos do TREC-COVID

In [ ]:
%%time
from pathlib import Path
import json

if not Path('./collections/trec-covid.zip').is_file():
  !wget {url_trec_covid} -P collections # type: ignore
  !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

# Converte o qrels que veio no trec-covid.zip pra o formato esperado:
with open('./collections/trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)
with open('./collections/trec-covid/qrels/test_corrigido.tsv', 'w') as fout:
  for linha in data[1:]:
    campos = linha.split()
    fout.write(f'{campos[0]}\t0\t{campos[1]}\t{campos[2]}\n')

def carrega_corpus_trec_covid():
  retorno = []
  with open('./collections/trec-covid/corpus.jsonl') as corpus:
    for i, line in enumerate(corpus):
      doc = json.loads(line)
      #retorno.append({
      #    'id': doc['_id'],
      #    'doc': f"{doc['title']} {doc['text']}"
      #})
      retorno.append(
          (doc['_id'], f"{doc['title']} {doc['text']}")
      )
      if (i % 10000 == 0):
        print(f'Processado {i} documentos')
    return retorno

def carrega_queries_trec_covid():
  retorno = []
  with open('./collections/trec-covid/queries.jsonl') as queries:
    for line in queries:
      query = json.loads(line)
      # Faz apenas uma pequena tradução de _id para id e text para texto
      retorno.append({'id': query['_id'], 'texto': query['text']})
  return retorno

queries_trec_covid = carrega_queries_trec_covid()
corpus_trec_covid = carrega_corpus_trec_covid()

Processado 0 documentos
Processado 10000 documentos
Processado 20000 documentos
Processado 30000 documentos
Processado 40000 documentos
Processado 50000 documentos
Processado 60000 documentos
Processado 70000 documentos
Processado 80000 documentos
Processado 90000 documentos
Processado 100000 documentos
Processado 110000 documentos
Processado 120000 documentos
Processado 130000 documentos
Processado 140000 documentos
Processado 150000 documentos
Processado 160000 documentos
Processado 170000 documentos
CPU times: user 1.48 s, sys: 60.4 ms, total: 1.54 s
Wall time: 1.52 s


In [ ]:
corpus_trec_covid[0]

('ug7v899j',
 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pn

## Recupera queries expandidas usando doc2query treinado na Aula 5

In [ ]:
from collections import OrderedDict
import pickle

dict_trec_covid_com_e_sem_expansao = OrderedDict({})
with open(arquivo_docs_queries_expandidas, 'rb') as f:
  dict_trec_covid_com_e_sem_expansao = pickle.load(f)

In [ ]:
# Apenas checa o formato dos dados (Dicionário com chave sendo a key do documento do TREC-COVID e o valor é um {doc_original: string, query_expandida: string})
dict_trec_covid_com_e_sem_expansao['mu5u5bvj']

{'doc_original': 'Relative cost and outcomes in the intensive care unit of acute lung injury (ALI) due to pandemic influenza compared with other etiologies: a single-center study BACKGROUND: Critical illness due to 2009 H1N1 influenza has been characterized by respiratory complications, including acute lung injury (ALI) or acute respiratory distress syndrome (ARDS), and associated with high mortality. We studied the severity, outcomes, and hospital charges of patients with ALI/ARDS secondary to pandemic influenza A infection compared with ALI and ARDS from other etiologies. METHODS: A retrospective review was conducted that included patients admitted to the Cleveland Clinic MICU with ALI/ARDS and confirmed influenza A infection, and all patients admitted with ALI/ARDS from any other etiology from September 2009 to March 2010. An itemized list of individual hospital charges was obtained for each patient from the hospital billing office and organized by billing code into a database. Cont

In [ ]:
ids_trec_covid, docs_trec_covid = list(dict_trec_covid_com_e_sem_expansao.keys()), list(dict_trec_covid_com_e_sem_expansao.values())
textos_trec_covid_expandidos = [f"{doc['doc_original']} {doc['query_expandida']}" for doc in docs_trec_covid]
textos_trec_covid_original = [doc['doc_original'] for doc in docs_trec_covid]
textos_trec_covid_so_expansao = [doc['query_expandida'] for doc in docs_trec_covid]

## Pipeline 1: doc2query + SPLADE

### Recuperando o código SPLADE (aula 7)

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from torch.nn.functional import relu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def carregar_tokenizador_e_modelo_splade(nome):
  tokenizer_splade = AutoTokenizer.from_pretrained(nome)
  model_splade = AutoModelForMaskedLM.from_pretrained(nome).to(device)

  return tokenizer_splade, model_splade

def representacao_esparsa_do_texto(model_splade, tokenizer_splade, texto, add_special_tokens=True, manter_contribuicao_cls_sep=False):
  # Tokeniza o texto
  # Para texto = "The quick brown jumps over the lazy dog",
  # tokens = ['the', 'quick', 'brown', 'jumps', 'over', 'the', 'lazy', 'dog'] (tamanho 8)
  # token_ids = [1996, 4248, 2829, 103, 14523, 2058, 1996, 13971, 3899] (tamanho 8)

  # Roda o modelo
  inputs = tokenizer_splade(texto, add_special_tokens=add_special_tokens,
                     return_special_tokens_mask=True,
                     return_tensors='pt',
                     truncation=True,
                     max_length=256)

  with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
    with torch.no_grad():
      outputs = model_splade(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))

  # Acessa os logits  
  # outputs.logits.size() = torch.Size([1, 8, 30522])
  # logits.size() = torch.Size([8, 30588])
  logits = outputs.logits[0, :]

  # Pelo artigo, agora a gente calcula somatório [ log(1 + ReLU(w_ij)) ]
  # relu(logits) vai manter o mesmo tamanho: [8, 30588]
  # 1 + relu(logits) também vai manter o mesmo tamanho: [8, 30588]
  # log(1 + relu(logits)) também vai manter o mesmo tamanho: [8, 30588]
  # Feito isso, calcula o somatório na dim=0, o que vai gerar um vetor de tamanho
  # 30588, que é o tamanho do vocabulário:
      
  mask_tokens_validos = 1 - inputs['special_tokens_mask'].to(device)
  mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(logits.size())

  if manter_contribuicao_cls_sep:
    mask = torch.ones(mask.size()).to(device) # Como não tem batch envolvido, isso é o mesmo que a attention mask

  if param_splade['agg'] == 'sum':
    wj = torch.sum(torch.log(1 + relu(logits*mask)), dim=0)
  else:
    wj, _ = torch.max(torch.log(1 + relu(logits*mask)), dim=0)

  # Agora temos que armazenar esse vetor de forma esparsa...
  return wj.to_sparse()

def converte_token_ids_para_tokens(tokenizer_splade, ids_tokens):
  return tokenizer_splade.convert_ids_to_tokens(ids_tokens)

In [ ]:
tokenizer_splade, model_splade = carregar_tokenizador_e_modelo_splade(param_splade['nome_modelo'])
model_splade.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
from torch.utils import data
from torch.utils.data import DataLoader

# Definição do Dataset
class DatasetSplade(data.Dataset):
    # Recebe apenas um vetor de textos
    def __init__(self, tokenizer_splade, textos, max_seq_length):
        self.max_seq_length = max_seq_length
        self.tokenizer_splade = tokenizer_splade
        self.textos = textos

    def __len__(self):
        return len(self.textos)
    
    def __getitem__(self, idx):
        # Aqui é só uma passada no eval, não precisa de cache
        item = self.tokenizer_splade(self.textos[idx],
                       padding=True,
                       return_special_tokens_mask=True,
                       # No exemplo dos autores eles não removem o CLS/SEP
                       # https://github.com/naver/splade/blob/main/inference_splade.ipynb
                       add_special_tokens=True, 
                       truncation=True,
                       max_length=self.max_seq_length
                )
        return item

In [ ]:
from transformers import BatchEncoding

def collate_fn_splade(batch):
    return BatchEncoding(tokenizer_splade.pad(batch, return_tensors='pt'))

In [ ]:
from tqdm.auto import tqdm

def representacao_esparsa_dataloader(model_splade, dataloader, func_executar_apos_batch = lambda idx_batch, wj_batch : None):
  with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
    with torch.no_grad():
      for i_batch, batch in enumerate(tqdm(dataloader)):
        outputs = model_splade(input_ids = batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device))
        logits = outputs.logits

        # Na hora de recuperar os logits, temos duas opções. Ou consideramos só a attention mask (ou seja, 
        # exclui apenas os PAD) ou consideramos a special tokens mask. Nessa situação remove (CLS, SEP e PAD).
        # Deixa configurável pra testar os dois
              # OBS.: No modelo do autor ele usa special_tokens e só remove o attention_mask. Vamos 
              # Mas tem uma classe SpaceDoc que parece que depois tira manualmente. Então vamos tentar simular
              # tirando os special_tokens_mask e depois se não der certo, tentamos tirar só o attention_mask
              # implementar assim também então: https://github.com/naver/splade/blob/main/splade/models/transformer_rep.py
        if param_splade['manter_contribuicao_CLS_SEP_da_matriz_doc']:
          mask_tokens_validos = batch['attention_mask'].to(device)
        else:
          mask_tokens_validos = 1 - batch['special_tokens_mask'].to(device)
        # Expande a máscara criando uma terceira dimensão (vocab_size) 
        # e colocando do mesmo tamanho que os logits (batch_size, x, vocab_size):
        mask = mask_tokens_validos.unsqueeze(-1).expand(logits.size())

        # Calcula a saída (os pesos wj)
        if param_splade['agg'] == 'sum':
          wj = torch.sum(torch.log(1 + relu(logits*mask)), dim=1)
        else:
          wj, _ = torch.max(torch.log(1 + relu(logits*mask)), dim=1)

        # Antes eu estava retornando wj.to_sparse() pois estava salvando
        # a matriz em disco. Pra salvar num índice invertido não tem mais
        # necessidade disso.

        # Callback
        idx_inicio = i_batch * dataloader.batch_size
        idx_fim = idx_inicio + min(dataloader.batch_size, wj.size()[0])
        indices_tratados = list(range(idx_inicio, idx_fim))
        func_executar_apos_batch(indices_tratados, wj)

In [ ]:
%%time
# Nesse ponto, textos_trec_covid_expandidos está pareado com a variável ids_trec_covid

dataset_trec_covid_expandido = DatasetSplade(tokenizer_splade, textos_trec_covid_expandidos, param_splade['max_seq_length'])
dataloader_trec_covid_expandido = DataLoader(dataset_trec_covid_expandido, batch_size=param_splade['batch_size'], shuffle=False, collate_fn=collate_fn_splade)

dataset_trec_covid_original = DatasetSplade(tokenizer_splade, textos_trec_covid_original, param_splade['max_seq_length'])
dataloader_trec_covid_original = DataLoader(dataset_trec_covid_original, batch_size=param_splade['batch_size'], shuffle=False, collate_fn=collate_fn_splade)

dataset_trec_covid_so_expansao = DatasetSplade(tokenizer_splade, textos_trec_covid_so_expansao, param_splade['max_seq_length'])
dataloader_trec_covid_so_expansao = DataLoader(dataset_trec_covid_so_expansao, batch_size=param_splade['batch_size'], shuffle=False, collate_fn=collate_fn_splade)

CPU times: user 211 µs, sys: 3 µs, total: 214 µs
Wall time: 221 µs


In [ ]:
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertidoSplade:

  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}

  def adiciona_docs(self, ids_docs, wjs_docs):
    nonzero = wjs_docs.nonzero()

    for i in range(wjs_docs.size()[0]):
      idx_linha_i = nonzero[:, 0] == i # i'ésimo doc do batch
      idx_token_em_wj_i = nonzero[idx_linha_i, 1] # além de ser o índice na matriz wj, é tb o id do token
      val_token_em_wj_i = wjs_docs[i, idx_token_em_wj_i]

      self.adiciona_doc(ids_docs[i], idx_token_em_wj_i, val_token_em_wj_i)

  def adiciona_doc(self, id_doc, idx_tokens, wj_tokens):
    for id, wj in zip(idx_tokens.tolist(), wj_tokens.tolist()):
      self.indice.setdefault(id, {"id_doc": [], "wj": array.array("f", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(id, {"id_doc": [], "wj": array.array("f", [])})['wj'].append(wj)
    
  def pesquisar(self, wjs_query, splade='v1'):
    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    wjs_query = wjs_query.coalesce()
    for id_token_query, wj_do_token_na_query in zip(wjs_query.indices()[0].tolist(), wjs_query.values().tolist()):
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer, apenas pula o termo
      if id_token_query not in self.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indice[id_token_query]['id_doc']
      wj_do_token_nos_docs = self.indice[id_token_query]['wj']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      multiplicador_token_query = wj_do_token_na_query if splade == 'v1' else 1

      for id_doc, wj_do_token_no_doc in zip(docs_que_tem_token, wj_do_token_nos_docs):
        docs_retornado_com_score[id_doc] += wj_do_token_no_doc * multiplicador_token_query
      
    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

In [ ]:
%%time
idx_splade = IndiceInvertidoSplade()

def popular_indice_invertido(idx_batch, wj_batch):
  ids_doc_batch = [ids_trec_covid[i] for i in idx_batch]
  idx_splade.adiciona_docs(ids_doc_batch, wj_batch)

def salvar_indice(idx_splade):
  nome_arquivo_pickle = param_splade["nome_arquivo_indice_invertido_docs"]
  diretorio_destino_cp = f"'{aula9_dir}'"
  with open(nome_arquivo_pickle, 'wb') as f:
    pickle.dump(idx_splade.indice, f)
  !cp {nome_arquivo_pickle} {diretorio_destino_cp}

def recuperar_indice(idx_splade):
  with open(arq_indice_pickle, 'rb') as f:
    idx_splade.indice = pickle.load(f)

arq_indice_pickle = f'{aula9_dir}{param_splade["nome_arquivo_indice_invertido_docs"]}'

if param_splade['gerar_arquivo_indice']:
  # Se for gerar o arquivo de índice, checar se é pra gerar usando doc2query ou não
  if param_splade['indexar_com_doc2query']:
    # Se for usar doc2query, checa se é pra usar só a expansão ou se é pra usar os doc originais + a expansão
    dataloader = dataloader_trec_covid_so_expansao if param_splade['indexar_so_expansao'] else dataloader_trec_covid_expandido
  else:
    dataloader = dataloader_trec_covid_original

  representacao_esparsa_dataloader(model_splade, dataloader, popular_indice_invertido)    
  salvar_indice(idx_splade)
else:
  recuperar_indice(idx_splade)

CPU times: user 4.11 s, sys: 201 ms, total: 4.31 s
Wall time: 4.31 s


### Roda queries

In [ ]:
import time

print('Carregando as queries do arquivo queries.jsonl...\n')
queries_trec_covid = carrega_queries_trec_covid()

def run_all_queries_indice_invertido_splade(file, model_splade, tokenizer_splade, idx_splade, splade='v1'):
  tempo_gpu = 0
  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id = query['id']
      texto = query['texto']

      tempo_inicio_splade = time.time()
      wj_query = representacao_esparsa_do_texto(model_splade, tokenizer_splade, texto, True, param_splade['manter_contribuicao_CLS_SEP_da_matriz_doc'])
      tempo_fim_splade = time.time()
      if cnt % 10 == 0:
        print(f'{cnt} queries completadas')

      # Usa o índice invertido pra pesquisar
      docs_score = idx_splade.pesquisar(wj_query, splade)

      for i in range(0, min(1000, len(docs_score))): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))

      cnt += 1
      tempo_gpu = tempo_gpu + (tempo_fim_splade - tempo_inicio_splade)

    print(f'{cnt} queries completadas')
    print(f"Tempo GPU: {tempo_gpu}")

Carregando as queries do arquivo queries.jsonl...



In [ ]:
%%time
run_all_queries_indice_invertido_splade('run-doc2query-splade.txt', model_splade, tokenizer_splade, idx_splade, 'v1')

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
Tempo GPU: 1.0339248180389404
CPU times: user 1min 10s, sys: 233 ms, total: 1min 10s
Wall time: 1min 9s


In [ ]:
%%time
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-doc2query-splade.txt

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-doc2query-splade.txt']
Results:
ndcg_cut_10           	all	0.7298
CPU times: user 52.6 ms, sys: 9.19 ms, total: 61.8 ms
Wall time: 5.53 s


## Pipeline 2: SPLADE + reranking InPars

### Recuperando (e adaptando) código reranking da Aula 8

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BatchEncoding
from transformers import AdamW
from torch.utils.data import DataLoader

# Carrega tokenizador e modelo e cria um optimizer
tokenizer_inpars = AutoTokenizer.from_pretrained(param_inpars['nome_tokenizer'])
model_inpars = AutoModelForSequenceClassification.from_pretrained(param_inpars['nome_modelo']).to(device)

# Cria dataloaders de treino e eval
collate_fn_inpars = lambda batch: BatchEncoding(tokenizer_inpars.pad(batch, return_tensors='pt'))

In [ ]:
class DatasetInpars(data.Dataset):
  # Recebe um dataframe do pandas. Precisa ter as colunas query, passage e label (0/1)
  def __init__(self, tokenizer_inpars, df, max_seq_length):
    self.max_seq_length = max_seq_length
    self.tokenizer_inpars = tokenizer_inpars

    # Já concatenas as query com as passagens e guarda em uma lista
    query_passage = df['query'] + ' [SEP] ' + df['passage']
    self.query_passage = query_passage.tolist()
    # Converte os labels para inteiros e guarda em uma lista
    self.labels = df.label.tolist()
    self.labels = [float(x) for x in self.labels]

    # Cria um cache vazio. Como tem treino em algumas épocas, guarda o encode no cache
    self.cache = {}

  def __len__(self):
    return len(self.query_passage)
  
  def get_token_type_ids(self, input_ids):
    idx_sep = input_ids.index(102)+1
    tam_seq = len(input_ids)
    token_type_ids = [0]*idx_sep + [1]*(tam_seq - idx_sep)

    # Apesar do tokenizer fazer isso, não precisa pois o attention_mask já zera.
    # for i in range(len(token_type_ids)):
    #   token_type_ids[i] = token_type_ids[i] if input_ids[i] != 0 else 0

    return token_type_ids

  def get_token_type_ids_from_slice(self, idx, matriz_input_ids):
    if isinstance(idx, slice):
      token_types = []
      for i in range(idx.start or 0, idx.stop or len(matriz_input_ids), idx.step or 1):
        token_types.append(self.get_token_type_ids(matriz_input_ids[i]))
      return token_types
    else:
      return self.get_token_type_ids(matriz_input_ids)

  def get_input_ids_e_labels(self, idx):
    input_ids_e_labels = self.tokenizer_inpars(self.query_passage[idx],
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_length)
    input_ids_e_labels['labels'] = self.labels[idx]

    input_ids_e_labels['token_type_ids'] = self.get_token_type_ids_from_slice(idx, input_ids_e_labels['input_ids'])

    return input_ids_e_labels

  def __getitem__(self, idx):
    # Guarda os itens tokenizados num dict e apenas recupera de lá, pra não ter que ficar tokenizando a cada época
    # Como estamos guardando no dict e idx é um slice, é necessário converter ele pra algo mapeável
    self.cache[str(idx)] = self.cache.get(str(idx), self.get_input_ids_e_labels(idx))
    return self.cache[str(idx)]

In [ ]:
%%time
from pyserini.search import get_qrels
import pandas as pd
import time

def reranking_docs(model_inpars, query, doc_ids):
  model_inpars.eval()
  
  # Cria um dataset para fazer o reranking
  if param_inpars['considerar_expansao_no_reranking']:
    passagens = [ f"{dict_trec_covid_com_e_sem_expansao[id]['doc_original']} {dict_trec_covid_com_e_sem_expansao[id]['query_expandida']}" for id in doc_ids ]
  else:
    passagens = [ dict_trec_covid_com_e_sem_expansao[id]['doc_original'] for id in doc_ids ] 

  queries = [query]*len(doc_ids)
  labels = [1]*len(doc_ids) # é eval, tanto faz o valor. Ele não será utilizado, mas precisamos cadastrar aqui
  df_reranking = pd.DataFrame({'query': queries, 'passage': passagens, 'label': labels})

  dataset_reranking = DatasetInpars(tokenizer_inpars, df_reranking, param_inpars['max_seq_length'])
  dataloader_reranking = DataLoader(dataset_reranking, batch_size=param_inpars['batch_size'], shuffle=False, collate_fn=collate_fn_inpars)

  score_todos_docs = []
  with torch.no_grad():
    for batch in dataloader_reranking:
      outputs = model_inpars(**batch.to(device))
      score = outputs.logits.squeeze().tolist()
      score_todos_docs.extend( score )

  docs_com_score = list(zip(doc_ids, score_todos_docs))

  torch.cuda.empty_cache()

  return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

# Roda todas as queries
def run_all_queries_splade_e_rerank(file, model_inpars, idx_splade, model_splade, tokenizer_splade, hits=1000):
  tempo_gpu = 0

  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(queries_trec_covid)))
    for query in queries_trec_covid:
      id = query['id']
      texto_query = query['texto']
      
      if cnt % 10 == 0:
        print(f'{cnt} queries completed')
      cnt += 1

      tempo_inicio_splade = time.time()
      wj_query = representacao_esparsa_do_texto(model_splade, tokenizer_splade, texto_query, True,
                                                param_splade['manter_contribuicao_CLS_SEP_da_matriz_doc'])
      tempo_fim_splade = time.time()
      # Usa o índice invertido Splade pra pesquisar
      docs_score = idx_splade.pesquisar(wj_query, 'v1')
      # Pega só os 1000 primeiros pra fazer o reranking
      n_reranking = min(hits, len(docs_score))
      docs_score = docs_score[0:n_reranking]

      # Agora faz o reranking com InPars
      docs, scores = zip(*docs_score)
      tempo_inicio_inpars = time.time()
      docs_score = reranking_docs(model_inpars, texto_query, docs)
      tempo_fim_inpars = time.time()
      
      tempo_gpu = tempo_gpu + (tempo_fim_inpars - tempo_inicio_inpars) + (tempo_fim_splade - tempo_inicio_splade)

      for i in range(0, len(docs_score)): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25_RERANKING_MINILM\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))

    print(f"Tempo GPU: {tempo_gpu}")



CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.6 µs


In [ ]:
%%time
run_all_queries_splade_e_rerank('run-splade-inpars.txt', model_inpars, idx_splade, model_splade, tokenizer_splade, hits=100)

Running 50 queries in total
0 queries completed


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


10 queries completed
20 queries completed
30 queries completed
40 queries completed
Tempo GPU: 22.814547300338745
CPU times: user 1min 28s, sys: 1.44 s, total: 1min 29s
Wall time: 1min 28s


In [ ]:
%%time
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-splade-inpars.txt #type: ignore     

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-splade-inpars.txt']
Results:
ndcg_cut_10           	all	0.7737
CPU times: user 52.2 ms, sys: 4.27 ms, total: 56.5 ms
Wall time: 5.23 s
